# Preparation

Preparation biasa dilakukan untuk mempersiapkan data sebelum masuk dalam tahap pemodelan. <br>
Berikut adalah tahapan yang akan dilalui pada data `SC_HW1_bank_data.csv` sebelum tahap pemodelan :
1. Import Library
2. Input Dataset
3. Preprocessing
4. Train-Test Split

## Import Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Input Dataset

In [ ]:
#Membaca data dan memasukkannya ke dalam bentuk Pandas Dataframe
df = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')

In [ ]:
#Jalankan code untuk mengecek nama kolom yang tersedia
df.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
#Hilangkan kolom yang dirasa tidak relevan dengan model (contoh: ID). None dapat diisi dengan nama-nama kolom yang akan digunakan.
#Contoh df = df[['X1','X2', 'Y']].copy()

#START CODE
df = df[['Geography', 'Age', 'Balance', 'IsActiveMember','Exited']].copy()

## Preprocessing

In [ ]:
#Lakukan One-Hot Encoder untuk data categorical, dengan fungsi pandas get_dummies

#START CODE
df = pd.get_dummies (df)

In [ ]:
#Pisahkan mana X (feature) dengan Y,
#Y adalah kolom "Exited"

#START CODE
X = df.drop (columns=["Exited"])
y = df['Exited']

In [ ]:
#Lakukan Scaler dan/atau Noermalisasi jika diperlukan
from sklearn.preprocessing import MinMaxScaler

#START CODE
scaler = MinMaxScaler ()
X_transform = scaler.fit_transform(X)

In [ ]:
#Ini digunakan jika dilakukan scaler/Normalisas. Jika tidak, code ini bisa dilewat dan diganti dengan code yang ada di dalam komen
X_transform = pd.DataFrame(X_transform, columns = X.columns)
#X_transform = X.copy()

## Train-Test Split

In [ ]:
#Split menjadi train dan test dengan test_size 25%
#Tidak perlu mengubah code ini

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_transform,y,test_size = 0.25,random_state = 123)

# Modeling

## Model1

### Soal :
Jelaskan secara Singkat Model pertama yang digunakan!

In [ ]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


#START CODE
!pip install catboost
from catboost import CatBoostClassifier
# Membuat model CatBoostClassifier
model = CatBoostClassifier(iterations=500,  # Jumlah iterasi (epoch)
                          learning_rate=0.1,  # Tingkat pembelajaran
                          depth=6,  # Kedalaman pohon
                          loss_function='Logloss',  # Fungsi loss
                          random_state=123,  # Seed untuk reproduksibilitas
                          verbose=100)  # Menampilkan informasi pelatihan setiap 100 iterasi
#END CODE

# Menentukan grid hyperparameter yang akan disetel
param_grid = {
    'n_estimators': [100, 200, 300],  # Jumlah pohon dalam ensemble
    'max_depth': [None, 10, 20, 30],  # Kedalaman maksimum pohon
    'min_samples_split': [2, 5, 10],  # Jumlah minimum sampel untuk membagi simpul
    'min_samples_leaf': [1, 2, 4]  # Jumlah minimum sampel di simpul daun
}

# Melatih model pada data latih
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

# Melakukan prediksi pada data uji
y_pred = model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

0:	learn: 0.6293951	test: 0.6297683	best: 0.6297683 (0)	total: 3ms	remaining: 1.5s
100:	learn: 0.3706573	test: 0.3892018	best: 0.3891964 (99)	total: 301ms	remaining: 1.19s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3890976015
bestIteration = 102

Shrink model to first 103 iterations.
Akurasi: 0.8376


In [ ]:
#lakukan evaluasi dengan beberapa maetric di bawah ini
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1983
           1       0.71      0.37      0.48       517

    accuracy                           0.84      2500
   macro avg       0.78      0.66      0.69      2500
weighted avg       0.82      0.84      0.82      2500


[[1904   79]
 [ 327  190]]

0.8376


## Model2
### Soal :
Jelaskan secara Singkat Model ke-2 yang digunakan!

In [ ]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


#START CODE
import xgboost as xgb
# Inisialisasi model XGBoost
model = xgb.XGBClassifier(random_state=123)  # Anda dapat menyesuaikan parameter sesuai kebutuhan

# Definisikan grid parameter yang akan diuji
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'min_child_weight': [1, 3, 5]
}

# Inisialisasi Grid Search dengan model dan parameter grid
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Melatih model dengan grid search
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
print("Hyperparameter Terbaik:", grid_search.best_params_)

# Membuat model dengan parameter terbaik
best_model = grid_search.best_estimator_

# Melatih model
model.fit(X_train, y_train)

# Melakukan prediksi
y_pred = model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi: {accuracy:.4f}')

Hyperparameter Terbaik: {'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 5, 'n_estimators': 100}
Akurasi: 0.8260


In [ ]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1983
           1       0.66      0.34      0.45       517

    accuracy                           0.83      2500
   macro avg       0.76      0.65      0.67      2500
weighted avg       0.81      0.83      0.81      2500


[[1894   89]
 [ 341  176]]

0.828


## Model3
### Soal :
Jelaskan secara Singkat Model ke-3 yang digunakan!

In [ ]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


#START CODE
!pip install lightgbm
import lightgbm as lgb
# Inisialisasi model LightGBM
model = lgb.LGBMClassifier()
#END CODE

# Menyusun parameter untuk model LightGBM (opsional)
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Melatih model
model.fit(X_train, y_train)

# Melakukan prediksi
y_pred = model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi: {accuracy:.4f}')

[LightGBM] [Info] Number of positive: 1520, number of negative: 5980
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000732 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 328
[LightGBM] [Info] Number of data points in the train set: 7500, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.202667 -> initscore=-1.369710
[LightGBM] [Info] Start training from score -1.369710
Akurasi: 0.8296


In [ ]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      1983
           1       0.67      0.35      0.46       517

    accuracy                           0.83      2500
   macro avg       0.76      0.65      0.68      2500
weighted avg       0.81      0.83      0.81      2500


[[1893   90]
 [ 336  181]]

0.8296


## Tarik Kesimpulan Model Mana yang terbaik beserta alasannya

In [ ]:
# Inisialisasi model-model yang akan diuji
models = [
    ('CatBoost', CatBoostClassifier()),
    ('XGBoost', xgb.XGBClassifier(random_state=123)),
    ('LightGBM', lgb.LGBMClassifier())
]

best_model = None
best_accuracy = 0

# Melakukan iterasi untuk menguji setiap model
for model_name, model in models:
    model.fit(X_train, y_train)  # Melatih model
    y_pred = model.predict(X_test)  # Melakukan prediksi
    accuracy = accuracy_score(y_test, y_pred)  # Menghitung akurasi

    print(f'{model_name} - Akurasi: {accuracy:.2f}')

    # Membandingkan dengan model terbaik sejauh ini
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model_name

print(f'Model terbaik adalah: {best_model} dengan akurasi {best_accuracy:.2f}')


Learning rate set to 0.024355
0:	learn: 0.6777730	total: 4.97ms	remaining: 4.97s
1:	learn: 0.6625991	total: 8.07ms	remaining: 4.03s
2:	learn: 0.6479215	total: 10.7ms	remaining: 3.57s
3:	learn: 0.6344549	total: 13.3ms	remaining: 3.32s
4:	learn: 0.6221243	total: 15.9ms	remaining: 3.16s
5:	learn: 0.6103790	total: 18.6ms	remaining: 3.08s
6:	learn: 0.5995758	total: 20.6ms	remaining: 2.93s
7:	learn: 0.5882309	total: 23.5ms	remaining: 2.92s
8:	learn: 0.5778671	total: 26.2ms	remaining: 2.88s
9:	learn: 0.5687688	total: 28.7ms	remaining: 2.84s
10:	learn: 0.5597296	total: 31.6ms	remaining: 2.85s
11:	learn: 0.5512227	total: 34.2ms	remaining: 2.82s
12:	learn: 0.5428634	total: 36.8ms	remaining: 2.79s
13:	learn: 0.5351856	total: 40ms	remaining: 2.81s
14:	learn: 0.5284574	total: 43.4ms	remaining: 2.85s
15:	learn: 0.5214725	total: 46.1ms	remaining: 2.83s
16:	learn: 0.5155624	total: 48.8ms	remaining: 2.82s
17:	learn: 0.5098196	total: 51.5ms	remaining: 2.81s
18:	learn: 0.5046643	total: 53.7ms	remaining: 